In [10]:
# Standard modules
import pdb
import os
import sys
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

sys.path.append('..')
from simstackwrapper import SimstackWrapper

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

# Import .fits table

In [2]:
from astropy.io import fits
path_cosmos2020 = os.path.join(os.environ['CATSPATH'], "cosmos","COSMOS2020_CLASSIC_R1_v2.0.fits")
if os.path.isfile(path_cosmos2020):
    print('Open ',path_cosmos2020)
    with fits.open(path_cosmos2020) as hdul:
        hdul.verify('fix')
        cosmos_catalog = hdul[1].data
else:
    print(path_cosmos2020, ' not found')

Open  D:\catalogs\cosmos\COSMOS2020_CLASSIC_R1_v2.0.fits


In [3]:
import pandas as pd
cosmos2020 = pd.DataFrame(cosmos_catalog)

In [4]:
cosmos2020.head()

,ID,ALPHA_J2000,DELTA_J2000,X_IMAGE,Y_IMAGE,ERRX2_IMAGE,ERRY2_IMAGE,ERRXY_IMAGE,FLUX_RADIUS,KRON_RADIUS,...,ez_ssfr_p025,ez_ssfr_p160,ez_ssfr_p500,ez_ssfr_p840,ez_ssfr_p975,ez_Av_p025,ez_Av_p160,ez_Av_p500,ez_Av_p840,ez_Av_p975
0,1,151.008534,1.398278,1304.953247,2579.132324,0.000009,0.000010,-8.461303e-07,NaN,NaN,...,3.073000e-14,3.306278e-14,3.696300e-14,3.828873e-14,3.828873e-14,0.704550,0.809278,0.896577,0.961172,1.070924
1,2,151.028825,1.404349,818.016785,2725.062988,0.000017,0.000017,-8.587199e-07,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,151.007706,1.398440,1324.814453,2583.007812,0.006722,0.008490,-7.216642e-04,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,150.162372,1.391619,21610.509766,2415.219971,0.031749,0.016943,6.141999e-06,NaN,NaN,...,1.191863e-10,1.356004e-10,1.582931e-10,1.872021e-10,2.076506e-10,1.659753,1.739529,1.869392,1.989442,2.089466
4,5,149.540181,1.391537,36540.632812,2414.936523,0.030666,0.016909,3.998309e-03,NaN,NaN,...,5.692379e-12,1.106644e-11,8.179705e-11,1.089352e-08,1.089352e-08,0.032678,0.032678,2.129271,3.005573,3.125110


# Convert to vaex for speed

In [5]:
import vaex
vaex_df = vaex.from_pandas(cosmos2020, copy_index=False)

In [6]:
# Export to hdf5 for more efficient use later
path_cosmos2020_hdf5 = os.path.join(os.environ['CATSPATH'], "cosmos","cosmos2020.hdf5")
vaex_df.export_hdf5(path_cosmos2020_hdf5)

In [7]:
# Convert flux to magnitude
# AB = -2.5 log S (Jy) + 8.9
vaex_df['rest_AB_U'] = (-2.5 * np.log10(vaex_df.ez_restU * 1e6) + 8.9)
vaex_df['rest_AB_V'] = (-2.5 * np.log10(vaex_df.ez_restV * 1e6) + 8.9)
vaex_df['rest_AB_J'] = (-2.5 * np.log10(vaex_df.ez_restJ * 1e6) + 8.9)

In [8]:
# Estimate rest-frame U-V and V-J
vaex_df['ez_restU-V']=(vaex_df.rest_AB_U - vaex_df.rest_AB_V) 
vaex_df['ez_restV-J']=(vaex_df.rest_AB_V - vaex_df.rest_AB_J)

In [9]:
# Extract subset of array and drop nans
features = ['ALPHA_J2000','DELTA_J2000','lp_zBEST','lp_mass_med','ez_restU-V','ez_restV-J','lp_sSFR_med','lp_type']
uvj_df_raw = vaex_df[features].dropna()
uvj_df = uvj_df_raw[uvj_df_raw['lp_type'] => 0]

SyntaxError: invalid syntax (<ipython-input-9-ac77dc7b17bc>, line 4)

In [ ]:
print(np.unique(uvj_df['lp_type'].values), ': unique values present in lp_type')
print(np.sum(uvj_df['lp_type'].values == -99), ' stars?')
print(np.sum(uvj_df['lp_type'].values == 0), ' galaxies')
print(np.sum(uvj_df['lp_type'].values == 1), ' agn')

In [ ]:
uvj_df.head()

# Export simplified, nan-free table, to csv

In [ ]:
path_cosmos2020_csv = os.path.join(os.environ['CATSPATH'], "cosmos","cosmos2020_UVJ.csv")
uvj_df.export_csv(path_cosmos2020_csv)

# Plot U-V/V-J vs specific star-formation rate

In [ ]:
sfr_cut = -9.5
ns=10000
uvj_df[:ns].scatter(uvj_df['ez_restV-J'],uvj_df['ez_restU-V'], selection = uvj_df['lp_sSFR_med'] > sfr_cut, alpha=0.25)
uvj_df[:ns].scatter(uvj_df['ez_restV-J'],uvj_df['ez_restU-V'], selection = uvj_df['lp_sSFR_med'] <= sfr_cut, alpha=0.25)